In [1]:
import pandas as pd
import numpy as np
import datetime   # datetime 내 timedelta는 몇주전후 까지만 나와 있음
from dateutil.relativedelta import relativedelta # 몇달전후, 몇년전후를 구하기 위함
from tqdm import tqdm
from scipy import stats

In [2]:
import weatherAPI 

In [7]:
other=pd.read_csv("month_other_weather.csv", encoding='utf-8-sig')

In [10]:
other.head()

,month,시도,avg_temp(C),daily_rain(mm),avg_wind(m/s),avg_r_humidity(%)
0,1,강원도,-4.4,0.2,1.4,60.0
1,1,경기도,-3.3,0.3,1.6,58.9
2,1,경상남도,0.1,0.4,1.6,54.5
3,1,경상북도,-0.3,0.6,2.3,50.7
4,1,부산광역시,3.1,0.5,3.3,41.8


In [8]:
occur=pd.read_csv("month_occur_weather.csv", encoding='utf-8-sig')

In [11]:
occur.head()

,month,시도,avg_temp(C),daily_rain(mm),avg_wind(m/s),avg_r_humidity(%),cnt
0,1,강원도,2.6,0.0,1.4,62.5,2
1,1,경기도,-3.4,0.6,1.6,63.2,66
2,1,경상남도,-1.0,0.0,1.8,54.0,12
3,1,경상북도,-0.9,0.1,1.7,50.2,3
4,1,부산광역시,6.1,0.0,1.1,58.0,2


#### RISK TABLE SET 제작
    1. occur과 non_occur과의 관계성이 나와있는 것만으로 table을 제작함
    2. 여름 5-9월 까지는 상대적으로 낮은 온도에서 발생하는 경향이 있음. (7~9월은 data 부족으로 판단)
       반면에 겨울에는 겨울 평상시 온도보다 약간 더 높은 온도에서 발생하는 경향이 있음 (12~2월) 
    3. 여름 5-9월 까지는 상대적으로 낮은 강수량일 경우 발생하는 경향이 있음. (7~9월은 data 부족으로 판단)
    4. 즉 12~2월은 온도가 겨울 평균온도보다 차이별로 높을경우 적용, 5~9월은 여름 평균온도보다 차이별로 낮을경우 적용

In [29]:
# 예시
occur.query("month==2")['avg_temp(C)'].mean()

3.227272727272728

In [30]:
other.query("month==2")['avg_temp(C)'].mean()

1.4363636363636363

In [37]:
month=[1,2,5,6,7,9,12]
cols_name=['month','avg_temp(C)','daily_rain(mm)','risk']
data=pd.DataFrame(columns=cols_name)
for mon in month:
    dic={}
    dic['month']=[mon,mon]
    avg_temp=np.round(other.query("month==@mon")['avg_temp(C)'].mean(),2)
    if mon in [5,6,7,9]:
        avg_rain=np.round(other.query("month==@mon")['daily_rain(mm)'].mean(),2)
        diff_temp=np.round(occur.query("month==@mon")['avg_temp(C)'].mean(),2)
        diff_rain=np.round(occur.query("month==@mon")['daily_rain(mm)'].mean(),2)
        dic['avg_temp(C)']=[avg_temp,diff_temp]
        dic['daily_rain(mm)']=[avg_rain, diff_rain]
    else:
        avg_rain=np.nan
        diff_temp=np.round(occur.query("month==@mon")['avg_temp(C)'].mean(),2)
        dic['avg_temp(C)']=[avg_temp,diff_temp]
        dic['daily_rain(mm)']=[avg_rain,avg_rain]
    dic['risk']=[0,10]
    temp=pd.DataFrame.from_dict(dic)
    data=pd.concat([data, temp], axis=0)
    data.reset_index(inplace=True, drop=True)

In [38]:
data

,month,avg_temp(C),daily_rain(mm),risk
0,1,-1.07,NaN,0
1,1,0.48,NaN,10
2,2,1.44,NaN,0
3,2,3.23,NaN,10
4,5,18.56,2.66,0
5,5,16.61,3.00,10
6,6,22.32,4.52,0
7,6,20.79,1.32,10
8,7,25.10,10.40,0
9,7,27.20,5.10,10


In [243]:
# 등분 기준 만들기
for n, group in tqdm(table.groupby(['month','시도'])):
    mon=n[0]
    sido=n[1]
    if sido=='대전광역시':  # 아예 없는 data
        sido='충청남도'
    occ=occur.query('month==@mon and 시도==@sido').reset_index(drop=True)
    oth=other.query('month==@mon and 시도==@sido').reset_index(drop=True)
    for col in cols_weather:
        if len(occ)!=0: # data가 있는 경우만..
            max_val=occ.loc[0,col]  # occur에 가까울수록 최고 점수
            min_val=oth.loc[0,col] # other에 가까울수록 최저 점수
            if max_val!=min_val:  # 두 data가 서로 같은 값이 아닌 경우
                table.loc[group.index[0],'index_'+col]= 10 # 최고 점수
                table.loc[group.index[0],col]=max_val
                table.loc[group.index[-1],'index_'+col]=0  # 최저 점수
                table.loc[group.index[-1],col]=min_val
                for index in group.index[1:-1]:
                    table.loc[index,'index_'+col]=table.loc[index-1,'index_'+col]-1
                    temp=np.round(table.loc[index-1,col]-((max_val-min_val)/len(group.index)), 3)  # 소수 셋째자리까지 등분
                    table.loc[index,col]=temp
            else: # 두 데이터 값이 서로 같은 경우는 가중치 0점 부여
                for index in group.index:
                    table.loc[index,'index_'+col]=0
                    table.loc[index,col]=max_val
    
        else: # data가 없는 경우.. 발생하지 않는다라고 가정!, 가중치 0점 부여
            for index in group.index:
                table.loc[index,'index_'+col]=0
                table.loc[index,col]=0  # 실제 값도 0으로 일단 제작

100%|████████████████████████████████████████████████████████████████████████████████| 204/204 [00:04<00:00, 50.83it/s]


In [244]:
# Risk 값 부여
for i in range(len(table)):
    cnt=0
    for col in cols_weight.keys():
        cnt+=table.loc[i,col]*cols_weight[col]
    table.loc[i,'Risk']=cnt

In [245]:
table.to_csv("Weather_Risk_table.csv", encoding='utf-8-sig', index=False)